# End to End Memory Network Experiment

In this experiment, I will base on the paper "End-to-end memory networks" to implement an `MemN2N` and test this network with a QA task, `bAbI v1.1`.

## Experiment Environment

- Colab
- Pytorch

## Data Exploration

Let's firs see the [bAbl](https://research.fb.com/downloads/babi/) dataset. The `bAbl` dataset contains 20 tasks for testing text understanding and reasoning.

The file format of each task is as follows:
```
ID text
ID text
ID text
ID question \t answer \t supporting fact IDs
```
For example:
```
1 Mary moved to the bathroom.
2 John went to the hallway.
3 Where is Mary?        bathroom        1
4 Daniel went back to the hallway.
5 Sandra moved to the garden.
6 Where is Daniel?      hallway 4
7 John moved to the office.
8 Sandra journeyed to the bathroom.
9 Where is Daniel?      hallway 4
10 Mary moved to the hallway.
11 Daniel travelled to the office.
12 Where is Daniel?     office  11
13 John went back to the garden.
14 John moved to the bedroom.
15 Where is Sandra?     bathroom        8
```
We can see that a given QA task contains a set of statements, followed by a question whose answer is typically a single word. Our job here is to let our network read the story and learn to answer the question.

We can first check our files.

In [1]:
import os
data_path = "/home/chiyeung/Datasets/bAbI/Tasks20/tasksv11/en"
data_dir = os.listdir(data_path)

In [2]:
sorted(data_dir)

['qa10_indefinite-knowledge_test.txt',
 'qa10_indefinite-knowledge_train.txt',
 'qa11_basic-coreference_test.txt',
 'qa11_basic-coreference_train.txt',
 'qa12_conjunction_test.txt',
 'qa12_conjunction_train.txt',
 'qa13_compound-coreference_test.txt',
 'qa13_compound-coreference_train.txt',
 'qa14_time-reasoning_test.txt',
 'qa14_time-reasoning_train.txt',
 'qa15_basic-deduction_test.txt',
 'qa15_basic-deduction_train.txt',
 'qa16_basic-induction_test.txt',
 'qa16_basic-induction_train.txt',
 'qa17_positional-reasoning_test.txt',
 'qa17_positional-reasoning_train.txt',
 'qa18_size-reasoning_test.txt',
 'qa18_size-reasoning_train.txt',
 'qa19_path-finding_test.txt',
 'qa19_path-finding_train.txt',
 'qa1_single-supporting-fact_test.txt',
 'qa1_single-supporting-fact_train.txt',
 'qa20_agents-motivations_test.txt',
 'qa20_agents-motivations_train.txt',
 'qa2_two-supporting-facts_test.txt',
 'qa2_two-supporting-facts_train.txt',
 'qa3_three-supporting-facts_test.txt',
 'qa3_three-supportin

In [3]:
data_1_path = os.path.join(data_path, data_dir[1])
print(f"File: {data_dir[1]}.\n")
with open(data_1_path, encoding='utf-8') as f:
    lines = f.readlines()
for line in lines[:9]:
    print(line)

File: qa10_indefinite-knowledge_train.txt.

1 Fred is either in the school or the park.

2 Mary went back to the office.

3 Is Mary in the office? 	yes	2

4 Bill is either in the kitchen or the park.

5 Fred moved to the cinema.

6 Is Fred in the park? 	no	5

7 Fred is in the office.

8 Bill moved to the cinema.

9 Is Bill in the cinema? 	yes	8



We can see that every question is just based on the sentences above.

## Data preprocessing

We first need a method to tokenize a sentence.

In [4]:
import re
def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

Then we need a method to read a specific task.

In [5]:
def load_tasks(data_dir, task_id):
    assert task_id > 0 and task_id < 21
    files = os.listdir(data_dir)
    files = [os.path.join(data_dir, f) for f in files]
    s = 'qa{}_'.format(task_id)
    
    # read the task_id train and test file
    train_files = [f for f in files if s in f and 'train' in f][0]
    test_files = [f for f in files if s in f and 'test' in f][0]
    
    # read train data and test data from two files
    train_data = get_stories(train_files)
    test_data = get_stories(test_files)
    
    return train_data, test_data

We also need a method to get the stories, queries and answers from the task file.

In [6]:
def get_stories(task_file):
    with open(task_file, encoding='utf-8') as f:
        lines = f.readlines()
    return parse_story(lines)

In [7]:
def parse_story(lines):
    data = []
    story = []
    
    for line in lines:
        line = line.lower()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            # This line is a question
            q, a, _ = line.split('\t')
            q = tokenize(q)
            a = [a]
            substory = None
            
            # remove question marks
            if q[-1] == '?':
                q = q[:-1]
            
            substory = [x for x in story if x]
            
            data.append((substory[::-1], q, a)) # reverse story
            story.append('')
        else:
            sent = tokenize(line)
            if sent[-1] == '.':
                sent = sent[:-1]
            story.append(sent)
    return data

We also need a method to load all the tasks together to form a big training data set or read the task one by one.

In [8]:
from itertools import chain
def load_data(data_dir, joint_training, task_num):
    if joint_training == 0:
        start_task = task_num
        end_task = task_num
    else:
        start_task = 1
        end_task = 20
    train_data = []
    test_data = []
    while start_task <= end_task:
        train_task, test_task = load_tasks(data_dir, task_num)
        train_data += train_task
        test_data += test_task
        start_task += 1
        
    data = train_data + test_data
    vocab = []
    for s, q, a in data:
        vocab += list(chain.from_iterable(s))
        vocab += q
        vocab += a
    vocab = list(set(vocab))
    return train_data, test_data, vocab

We also need a method to transform a word sentence to a index sentence.

In [9]:
def word_to_index(sent, w2i):
    """
    sent: a word sentence
    w2i: a mapping, word -> index
    """
    vec = []
    for w in sent:
        if w in w2i:
            vec.append(w2i[w])
        else:
            vec.append(w2i['<PAD>'])
    return vec

We also need a function to vectorize all the data, pad all the stories to the same length, pad all the storyies' sentences to the same length and pad all the queries to the same length.

In [10]:
def vectorize(data, w2i, story_len, sent_len, q_len):
    vec_data = []
    for d in data:
        tmp_story = d[0]
        story = []
        for s in tmp_story:
            sent = word_to_index(s, w2i)
            if len(sent) < sent_len:
                sent += [0] * (sent_len - len(sent))
            story.append(sent)
        while len(story) < story_len:
            story.append([0] * sent_len)
        story = story[:story_len]
        q = d[1]
        q = word_to_index(q, w2i)
        if len(q) < q_len:
            q += [0] * (q_len - len(q))
        a = d[2]
        a = word_to_index(a, w2i)
        vec_data.append((story, q, a))
    return vec_data

## Model Arcitecture

In this section, I will build the class of MemN2N.

In [11]:
import torch
import torch.nn as nn

class MemN2N(nn.Module):
    def __init__(self, vocab_size, embed_size, story_len, device, hops=3, dropout_rate=0.2):
        super(MemN2N, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.device = device
        self.hops = hops
        self.dropout = nn.Dropout(p=dropout_rate)
        self.A = nn.ModuleList([nn.Embedding(vocab_size, embed_size) for _ in range(hops+1)])
        for i in range(len(self.A)):
            self.A[i].weight.data.normal_(0, 0.1)
            self.A[i].weight.data[0] = 0
        self.B = self.A[0]
        self.TA = nn.Parameter(torch.Tensor(1, story_len + 1, embed_size).normal_(0, 0.1))
        self.TC = nn.Parameter(torch.Tensor(1, story_len + 1, embed_size).normal_(0, 0.1))
        self.proj = nn.Linear(embed_size, vocab_size)
    
    def forward(self, x, q):
        """
        x: input story, [batch_size, story_len, sent_len]
        q: query, [batch_size, q_len]
        """
        batch_size, story_len, sent_len = x.shape

        # positional encoding
        J = sent_len
        d = self.embed_size
        position = torch.arange(0, J).reshape(-1, 1)
        pe = (1 - position / J) - (torch.arange(1, d+1) / d) * (1 - 2 * position / J)
        pe = pe.to(device, torch.float) # [sent_len, embed_size]
        pe = pe.unsqueeze(0).unsqueeze(0) # [1, 1, sent_len, embed_size]
        pe = pe.repeat(batch_size, story_len, 1, 1) # [batch_size, story_len, sent_len, embed_size]
        
        # query embedding
        u = self.dropout(self.B(q)) # [batch_size, q_len, embed_size]
        u = u.sum(dim=1) # [batch_size, embed_size]
        
        for k in range(self.hops):
            #---- m ----
            m = self.dropout(self.A[k](x)) # [batch_size, story_len, sent_len, embed_size]
            m = m * pe # [batch_size, story_len, sent_len, embed_size]
            m = m.sum(dim=2) # [batch_size, story_len, embed_size]
            m += self.TA.repeat(batch_size, 1, 1)[:, :story_len, :] # [batch_size, story_len, embed_size]
            
            #---- p ----
            u = u.unsqueeze(1) # [batch_size, 1, embed_size]
            p = u * m # [batch_size, story_len, embed_size]
            p = p.sum(dim=2) # [batch_size, story_len]
            p = torch.softmax(p, dim=-1)
            u = u.squeeze(1) # [batch_size, embed_size]
            
            #---- c ----
            c = self.dropout(self.A[k+1](x)) # [batch_size, story_len, sent_len, embed_size]
            c *= pe # [batch_size, story_len, sent_len, embed_size]
            c = c.sum(dim=2) # [batch_size, story_len, embed_size]
            c += self.TC.repeat(batch_size, 1, 1)[:, :story_len, :] # [batch_size, story_len. embed_size]
            
            #---- o ----
            p = p.unsqueeze(2) # [batch_size, story_len, 1]
            o = p * c # [batch_size, story_len, embed_size]
            o = o.sum(dim=1) # [batch_size, embed_size]
            
            #---- new u ----
            u = u + o # [batch_size, embed_size]
        
        # Linear projection
        out = self.proj(u) # [batch_size, vocab_size]
        return out

## Model Traning Function

In [12]:
import random
def train(model, train_data, batch_size, num_epochs, optimizer,
          criterion, w2i, max_story_len, device):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.
        random.shuffle(train_data)
        for i in range(0, len(train_data)-batch_size, batch_size):
            batch_data = train_data[i:i+batch_size]
            story = [d[0] for d in batch_data]
            # compute the number of story
            story_len = min(max_story_len, max([len(s) for s in story]))
            # compute the length of each sentence in a story
            s_sent_len = max([len(sent) for s in story for sent in s])
        
            # compute query sentence length
            query = [d[1] for d in batch_data]
            q_sent_len = max([len(q) for q in query])

            # Then we need to vectorize all the data
            vec_data = vectorize(batch_data, w2i, story_len, s_sent_len, q_sent_len)
            story = torch.LongTensor([d[0] for d in vec_data]).to(device)
            query = torch.LongTensor([d[1] for d in vec_data]).to(device)
            a = torch.LongTensor([d[2][0] for d in vec_data]).to(device)
        
            # model prediction
            pred = model(story, query)
        
            # loss computation
            loss = criterion(pred, a)
        
            # optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            # reset padding index weight
            for name, param in model.named_parameters():
                if param.grad is not None:
                    if 'A.' in name:
                        param.data[0] = 0
        
            # gradient clipping
            for p in model.parameters():
                torch.nn.utils.clip_grad_norm_(p, 40.0)
            
            total_loss += loss.item()
        if (epoch+1) % 20 == 0:
            print(f"Epoch:{epoch+1}. Loss:{total_loss/batch_size:.2f}.")
            total_loss = 0.

## Model Testing function

In [13]:
def test(model, test_data, batch_size, w2i, max_story_len, device):
    model.eval()
    correct = 0
    total = 0
    for i in range(0, len(test_data)-batch_size, batch_size):
        batch_data = test_data[i:i+batch_size]
        
        # compute the number of story
        story = [d[0] for d in batch_data]
        story_len = min(max_story_len, max([len(s) for s in story]))
        
        # compute the length of a sentence of a story
        s_sent_len = max([len(sent) for s in story for sent in s])
        
        # comptue the length of a query
        query = [d[1] for d in batch_data]
        q_sent_len = max([len(q) for q in query])
        
        # Then we vectorize all the data
        vec_data = vectorize(batch_data, w2i, story_len, s_sent_len, q_sent_len)
        story = torch.LongTensor([d[0] for d in vec_data]).to(device)
        query = torch.LongTensor([d[1] for d in vec_data]).to(device)
        a = torch.LongTensor([d[2][0] for d in vec_data]).to(device)
        
        # prediction
        pred = model(story, query)
        _, pred = pred.max(dim=-1)
        correct += (pred==a).sum().item()
        total += len(batch_data)
    return correct / total

## Model Running function:

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyparameter
num_epochs = 200
max_story_len = 25
embed_size = 30
batch_size = 32

# criterion
criterion = nn.CrossEntropyLoss()

def run():
    test_acc_results = []
    for task_id in range(1, 21):
        print(f"==== Task {task_id} ====")
        train_data, test_data, vocab = load_data(data_path, 0, task_id)
        w2i = dict((w, i) for i, w in enumerate(vocab, 1))
        w2i['<PAD>'] = 0
        vocab_size = len(w2i)
        story_len = min(max_story_len, max([len(s) for s, q, a in train_data + test_data]))
        model = MemN2N(vocab_size, embed_size, story_len, device).to(device)
        optimizer = torch.optim.Adam(model.parameters())
        train(model, train_data, batch_size, num_epochs, optimizer,
              criterion, w2i, story_len, device)
        acc = test(model, test_data, batch_size, w2i, story_len, device)
        print(f"Task {task_id} test acc: {acc:.2%}.")
        test_acc_results.append(acc)
    return test_acc_results

## Result

In [15]:
test_acc_res = run()

==== Task 1 ====


/home/chiyeung/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Epoch:20. Loss:0.22.
Epoch:40. Loss:0.12.
Epoch:60. Loss:0.08.
Epoch:80. Loss:0.07.
Epoch:100. Loss:0.03.
Epoch:120. Loss:0.04.
Epoch:140. Loss:0.02.
Epoch:160. Loss:0.02.
Epoch:180. Loss:0.02.
Epoch:200. Loss:0.02.
Task 1 test acc: 100.00%.
==== Task 2 ====
Epoch:20. Loss:1.44.
Epoch:40. Loss:1.26.
Epoch:60. Loss:1.11.
Epoch:80. Loss:0.95.
Epoch:100. Loss:0.77.
Epoch:120. Loss:0.61.
Epoch:140. Loss:0.50.
Epoch:160. Loss:0.46.
Epoch:180. Loss:0.36.
Epoch:200. Loss:0.29.
Task 2 test acc: 65.83%.
==== Task 3 ====
Epoch:20. Loss:1.42.
Epoch:40. Loss:1.12.
Epoch:60. Loss:0.99.
Epoch:80. Loss:0.83.
Epoch:100. Loss:0.78.
Epoch:120. Loss:0.69.
Epoch:140. Loss:0.73.
Epoch:160. Loss:0.63.
Epoch:180. Loss:0.60.
Epoch:200. Loss:0.54.
Task 3 test acc: 42.24%.
==== Task 4 ====
Epoch:20. Loss:0.90.
Epoch:40. Loss:0.64.
Epoch:60. Loss:0.60.
Epoch:80. Loss:0.55.
Epoch:100. Loss:0.52.
Epoch:120. Loss:0.52.
Epoch:140. Loss:0.53.
Epoch:160. Loss:0.51.
Epoch:180. Loss:0.50.
Epoch:200. Loss:0.49.
Task 4 te

In [16]:
import pandas as pd

task = []

for i in range(1, 21):
    task_id = "task {}".format(i)
    task.append(task_id)

df = pd.DataFrame(test_acc_res, index=task, columns=['acc'])
df

,acc
task 1,1.000000
task 2,0.658266
task 3,0.422379
task 4,0.752016
task 5,0.812500
task 6,0.843750
task 7,0.792339
task 8,0.872984
task 9,0.851815
task 10,0.826613
